In [14]:
#alte version, json files waren zu groß
import json
import os
#alle Spiele = ["Counter-Strike_2","Raft", "Rust", "DayZ", "Payday_2", "Insurgency"]
paths = ["Rust", "DayZ"]

# Ordner mit JSON-Dateien
base_path = r"C:\Users\paulu\OneDrive\Dokumente\GitHub\WebMining\Paul\Analyse"

# Liste für die kombinierten Daten
combined_data = []

# Durchlaufe alle angegebenen Spielordner
for game_folder in paths:
    folder_path = os.path.join(base_path, game_folder)
    
    # Prüfen, ob der Ordner existiert
    if not os.path.isdir(folder_path):
        print(f"Ordner '{folder_path}' existiert nicht. Überspringe...")
        continue

    # Durchlaufe alle JSON-Dateien im jeweiligen Ordner
    for filename in os.listdir(folder_path):
        if filename.endswith('.json'):
            file_path = os.path.join(folder_path, filename)
            
            # 1) Steam-ID aus dem Dateinamen extrahieren
            # Beispiel: "Counter-Strike_2_76561197960716112.json"
            # -> split über "_" und nimm das letzte Element (ohne .json)
            base_name = filename[:-5]  # entfernt ".json"
            parts = base_name.split('_')  
            steam_id = parts[-1]  # Das letzte Element als ID

            # 2) JSON-Datei laden
            with open(file_path, 'r', encoding='utf-8') as file:
                data = json.load(file)

            # 3) Struktur ermitteln (Dictionary oder Liste)
            if isinstance(data, dict):  # mit playerstats -> stats
                stats = data.get('playerstats', {}).get('stats', [])
            elif isinstance(data, list):  # falls direkt eine Liste
                stats = data
            else:
                # Unerwartete Struktur -> Überspringen
                continue

            # 4) Felder "name", "value" und neu: "steam_id" übernehmen
            for stat in stats:
                if 'name' in stat and 'value' in stat:
                    combined_data.append({
                        "steam_id": steam_id,
                        "name": stat["name"],
                        "value": stat["value"]
                    })

# 5) Speichere die kombinierten Daten in einer einzigen JSON-Datei
output_file = rf'combined_data_{game_folder}.json'
with open(output_file, 'w', encoding='utf-8') as outfile:
    json.dump(combined_data, outfile, ensure_ascii=False, indent=4)

print(f"Kombinierte Daten inkl. 'steam_id' wurden in '{output_file}' gespeichert.")

Kombinierte Daten inkl. 'steam_id' wurden in 'combined_data_DayZ.json' gespeichert.


In [1]:
import json
import os
#alle Spiele = ["Counter-Strike_2","Raft", "Rust", "DayZ", "Payday_2", "Insurgency"]
paths = ["Counter-Strike_2","Raft", "Rust", "DayZ", "Payday_2", "Insurgency"]
# Ordner mit JSON-Dateien
base_path = r"C:\Users\paulu\OneDrive\Dokumente\GitHub\WebMining\Paul\Analyse"

list_json_count =[]
# Zähle die Anzahl der JSON-Dateien im Ordner
for game_folder in paths:
    folder_path = os.path.join(base_path, game_folder)
    json_file_count = len([filename for filename in os.listdir(folder_path) if filename.endswith('.json')])
    game_name = game_folder.replace("_", " ")
    list_json_count.append(json_file_count)
    print(f"{game_name}: {json_file_count}")

Counter-Strike 2: 9383
Raft: 2101
Rust: 3585
DayZ: 1395
Payday 2: 6005
Insurgency: 1346


In [2]:
import os
import json
import pandas as pd

games = [
    {"appid": 730, "name": "Counter-Strike_2"},
    {"appid": 648800, "name": "Raft"},
    {"appid": 221100, "name": "DayZ"},
    {"appid": 222880, "name": "Insurgency"},
    {"appid": 218620, "name": "Payday_2"},
    {"appid": 252490, "name": "Rust"}
]

paths = ["Payday_2"]

# Hauptordner, in dem sich die Spiele-Ordner befinden (ggf. anpassen)
base_path = r"C:\Users\paulu\OneDrive\Dokumente\GitHub\WebMining\Paul\Analyse"

for game_folder in paths:
    folder_path = os.path.join(base_path, game_folder)
    
    # Prüfe, ob der Ordner existiert
    if not os.path.isdir(folder_path):
        print(f"Ordner '{folder_path}' existiert nicht. Überspringe...")
        continue
    
    # Liste, um Daten aus allen JSON-Dateien zu sammeln
    data_list = []
    
    # Durchlaufe alle JSON-Dateien im jeweiligen Ordner
    for filename in os.listdir(folder_path):
        if filename.endswith('.json'):
            file_path = os.path.join(folder_path, filename)
            
            # Aus dem Dateinamen die Steam-ID extrahieren (Beispiel: "Raft_76561197960716112.json")
            base_name = filename[:-5]  # ".json" abschneiden
            parts = base_name.split('_')
            # Wenn das Format immer "Spielname_<SteamID>.json" ist:
            steam_id = parts[-1]  # das letzte Element als ID
            
            # JSON-Datei laden
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
            
            # Bestimme, wie die Stats-Struktur aussieht:
            if isinstance(data, dict):
                stats = data.get('playerstats', {}).get('stats', [])
            elif isinstance(data, list):
                stats = data
            else:
                # Unerwartete Struktur -> Überspringen
                continue
            
            # Felder "name" und "value" erfassen
            for stat in stats:
                if 'name' in stat and 'value' in stat:
                    data_list.append({
                        "steam_id": steam_id,
                        "name": stat["name"],
                        "value": stat["value"]
                    })
    
    # Falls keine Daten gefunden wurden, weiter zum nächsten Spiel
    if not data_list:
        print(f"Keine Daten in Ordner '{game_folder}' gefunden.")
        continue
    
    # DataFrame erstellen
    df = pd.DataFrame(data_list)  # enthält Spalten: steam_id, name, value
    
    # Pivot-Tabelle erstellen
    pivot_df = df.pivot(index='steam_id', columns='name', values='value')
    
    # Finde die appid für das aktuelle Spiel
    appid = next((game['appid'] for game in games if game['name'] == game_folder), None)
    
    # CSV-Datei je Spielordner speichern
    output_csv = f"{appid}_stats_{game_folder}.csv" if appid else f"stats_{game_folder}.csv"
    pivot_df.to_csv(output_csv, index=True)  # index=True, damit steam_id in der CSV sichtbar bleibt
    
    print(f"CSV für '{game_folder}' wurde erstellt: {output_csv}")


CSV für 'Payday_2' wurde erstellt: 218620_stats_Payday_2.csv


In [41]:
game_stats_data = ["730_stats_Counter-Strike_2.csv", "648800_stats_Raft.csv", "252490_stats_Rust.csv" ,"221100_stats_DayZ.csv", "218620_stats_Payday_2.csv", "222880_stats_Insurgency.csv"]

list_csv_count =[]

#print amount of rows for each file
for file in game_stats_data:
    df = pd.read_csv(file)
    game_name = file.split('_')[-1].split('.')[0]
    csv_count = len(df)
    list_csv_count.append(csv_count)
    #duplicate_count = df.duplicated().sum()
    #print(f"{file}: {len(df)} Zeilen, {duplicate_count} Duplikate")

# Calculate the difference between the values in list_csv_count and list_json_count
list_difference = [csv - json for csv, json in zip(list_csv_count, list_json_count)]

print(list_csv_count)
print(list_json_count)
print(list_difference)






[9331, 1182, 1737, 295, 2674, 800]
[9383, 2101, 3585, 1395, 6005, 1346]
[-52, -919, -1848, -1100, -3331, -546]
